In [1]:
CLUSTER_TYPE ='docker_cluster'

In [2]:
import os

KAFKA_BOOTSTRAP_SERVERS = ''

if CLUSTER_TYPE == 'local':

    KAFKA_HOME = '~/Downloads/kafka_2.13-2.8.0'
    KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092']
    
    # Start Zookeeper    
    os.system('{0}/bin/zookeeper-server-start.sh {0}/config/zookeeper.properties'.format(KAFKA_HOME)) 
    
    # Start one Kafka Broker
    os.system('{0}/bin/kafka-server-start.sh {0}/config/server.properties'.format(KAFKA_HOME)) 
    
elif CLUSTER_TYPE == 'docker_cluster':

    KAFKA_BOOTSTRAP_SERVERS = ['kafka-broker:9092']

In [3]:
import csv
import dask.array as da
import numpy as np
from time import sleep
import threading
rexp = np.random.exponential

def data_post(file,rate,log):
    with open(file,newline = '') as file:
        #times = da.random.exponential(1/rate,12000).compute()
        data = csv.DictReader(file)
        for i, entry in enumerate(data):
            #sleep(times[i])
            sleep(max(rexp(1/rate)-0.00050,0))# a lot of time is wasted elsewhere, "one by one" rates are hard to achieve
            #x = threading.Thread(target = log, args = (entry,))
            #x.start()
            log(entry)

In [4]:
%%time
sample_file = 'sample_data.csv'
entries_per_sec = 300

data_post(sample_file, entries_per_sec, lambda x: None)

CPU times: user 1.61 s, sys: 227 ms, total: 1.84 s
Wall time: 38.2 s


In [5]:
from kafka import KafkaProducer
from json import dumps

In [19]:
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                         api_version=(0,11,5),
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'),
                        )

In [17]:
producer.send(topic, b'test')

KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.

In [11]:
topic = 'raw-data'
def logger(x):
    producer.send(topic, value = x)
    producer.flush()

data_post(sample_file, entries_per_sec, logger)

KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.